<h2 style="text-align:center;font-size:200%;;">Vietnamese Sentiment Analyais </h2>
<h3  style="text-align:center;"><span class="label label-success">NLP</span> <span class="label label-success">Sentiment Analysis</span> <span class="label label-success">Bag of Words</span> <span class="label label-success">TF IDF</span> <span class="label label-success">Basic Machine Learning</span> <span class="label label-success"></span></h3>

<a class="anchor" id="0.1"></a>
[](http://)

# **Table of Contents**


1.	[READ DATA](#1)
2.	[IMPLEMENT MODEL USING BAG OF WORDS](#2)
3.  [Naive bayes bow](#3)
4.	[Logistic regression bow](#4)
5.	[Decision Tree bow](#5)    
6.	[KNN bow](#6)
7.	[SVM bow](#7)
10.	[IMPLEMENT MODEL USING TF-IDF](#8)
11.	[Naive bayes tf-idf](#9)
12.	[Logistic regression tf-idf](#10)
13.	[Decision Tree tf-idf](#11)
14.	[KNN tf-idf](#12)
15.	[SVM tf-idf](#13)

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint
from pprint import pprint
import tqdm
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn import tree
!pip install pyvi
from pyvi import ViTokenizer
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tientd3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


You should consider upgrading via the 'c:\users\tientd3\vinbdi\scripts\python.exe -m pip install --upgrade pip' command.


# **1. Read data** <a class="anchor" id="1"></a>

[Go back to table of contents](#0.1)

In [61]:
# Read data
df_train = pd.read_csv('train.csv')
df_valid = pd.read_csv('eval.csv')
df_test = pd.read_csv('test.csv')
# Read stopword
df_stopword = pd.read_csv('vietnamese-stopwords-dash.txt', header=None)
stopword = df_stopword.iloc[:,0].values

Check data distribution

In [10]:
print('df_train length distribution')
len_train = df_train.comment.str.len()
len_train.mean(), len_train.std(), len_train.max(), len_train.min()

df_train length distribution


(49.543780303956325, 41.605379552354975, 378, 5)

In [11]:
print('df_valid length distribution')
len_valid = df_valid.comment.str.len()
len_valid.mean(), len_valid.std(), len_valid.max(), len_valid.min()

df_valid length distribution


(48.27607561929596, 40.24405155635034, 323, 6)

In [12]:
print('df_test length distribution')
len_test = df_test.comment.str.len()
len_test.mean(), len_test.std(), len_test.max(), len_test.min()

df_test length distribution


(48.67731421121252, 41.112807906276714, 343, 6)

In [14]:
# Check labels percentage
df_train.label.value_counts(normalize=True)

POS    0.639368
NEG    0.212769
NEU    0.147863
Name: label, dtype: float64

In [15]:
dic = {'NEG':0, 'NEU':1, 'POS':2}

def convert_df(df, dic):
    x = df.comment.values
    y = df.label.map(dic)
    return x, y

xtrain, ytrain = convert_df(df_train, dic)
xvalid, yvalid = convert_df(df_valid, dic)
xtest, ytest = convert_df(df_test, dic)
print(xtrain)
print(ytrain)

['vải đẹp  kiểu ih hình   rất hài lòng ' 'màn hình to và khá là hd '
 'đóng gói sản phẩm rất đẹp và chắc chắn chất lượng sản phẩm tuyệt vời nhưng hơi không vừa vì hơi rộng '
 ...
 'đẹp lắm  cửa hàng  ơi  mình mập cứ sợ không vừa  ai ngờ vừa y  cảm ơn  cửa hàng  '
 'chất lượng sản phẩm tuyệt vời ' 'ko  được  sao nào hết ']
0        2
1        2
2        2
3        2
4        2
        ..
24538    1
24539    0
24540    2
24541    2
24542    0
Name: label, Length: 24543, dtype: int64


## **2 Implement Models Using Bag of words** <a class="anchor" id="2"></a>

[Go back to table of contents](#0.1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
ctv = CountVectorizer(tokenizer = word_tokenize,stop_words=list(stopword),
                       token_pattern = None)
ctv.fit(xtrain)
xtrain_transform, xvalid_transform, xtest_transform = ctv.transform(xtrain), ctv.transform(xvalid), ctv.transform(xtest)

### **2.1. Naive bayes BOW** <a class="anchor" id="3"></a>

[Go back to table of contents](#0.1)

In [20]:
import pickle
from sklearn.naive_bayes import MultinomialNB
model_naivebayes = MultinomialNB()
model_naivebayes.fit(xtrain_transform, ytrain)
preds_train = model_naivebayes.predict(xtrain_transform)
preds_valid = model_naivebayes.predict(xvalid_transform)
preds_test =  model_naivebayes.predict(xtest_transform)
print('Naive BS accuracy train')
print(metrics.accuracy_score(ytrain, preds_train))
print('Naive BS accuracy valid')
print(metrics.accuracy_score(yvalid, preds_valid))
print('Naive BS accuracy test')
print(metrics.accuracy_score(ytest, preds_test))

Pkl_Filename = "MultinomialNB_bow.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model_naivebayes, file)

Naive BS accuracy train
0.7661247606242105
Naive BS accuracy valid
0.7330508474576272
Naive BS accuracy test
0.7340286831812256


### **2.2. Logistic regression BOW** <a class="anchor" id="4"></a>

[Go back to table of contents](#0.1)

#### Cách nhóm em tunning hyperparameters cho các mô hình :
#### Sử dụng sự kết hợp giữa random search + gridsearch để tìm ra hyperparameter tốt 
#### Bước 1: Sử dụng random search để tìm kiếm trên diện rộng 1 cách ngẫu nhiên. Giá trị trả về là parameter tốt nhất theo random search.
#### Bước 2: Từ giá trị trả về ở bước 1, ta biết được parameters cần tìm sẽ nằm ở khoảng lân cận gần đó. Lúc này dùng gridsearch để tìm kiếm trong các giá trị lân cận này

In [57]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Tuning C penalty
# Random search
c = np.logspace(-4,4,10)
grid_cv =    {
               'C': c,
               }

lr = linear_model.LogisticRegression(n_jobs=-1, penalty='l2')  # solver: saga

model =RandomizedSearchCV(     estimator = lr, 
                               param_distributions= grid_cv, 
                               scoring='accuracy',
                               cv = 5,
                               n_jobs = -1
                    )

model.fit(xtrain_transform, ytrain)

print(f'Best score: {model.best_score_}')
print(f'Best parameters set: ')
best_parameters = model.best_estimator_.get_params()
for pagram in best_parameters.keys():
    print(f'\t{pagram}: {best_parameters[pagram]}')

Best score: 0.7548385838229701
Best parameters set: 
	C: 0.3593813663804626
	class_weight: None
	dual: False
	fit_intercept: True
	intercept_scaling: 1
	l1_ratio: None
	max_iter: 100
	multi_class: auto
	n_jobs: -1
	penalty: l2
	random_state: None
	solver: lbfgs
	tol: 0.0001
	verbose: 0
	warm_start: False


In [59]:
# After we see C = 0.36, gridsearch
c = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

grid_cv =    {
               'C': c,
               }

lr = linear_model.LogisticRegression(n_jobs=-1, penalty='l2')  # solver: saga

model = GridSearchCV(          estimator = lr, 
                               param_grid = grid_cv, 
                               scoring='accuracy',
                               cv = 5,
                               n_jobs = -1
                    )

model.fit(xtrain_transform, ytrain)

print(f'Best score: {model.best_score_}')
print(f'Best parameters set: ')
best_parameters = model.best_estimator_.get_params()
for pagram in best_parameters.keys():
    print(f'\t{pagram}: {best_parameters[pagram]}')

Best score: 0.7551237078811551
Best parameters set: 
	C: 0.2
	class_weight: None
	dual: False
	fit_intercept: True
	intercept_scaling: 1
	l1_ratio: None
	max_iter: 100
	multi_class: auto
	n_jobs: -1
	penalty: l2
	random_state: None
	solver: lbfgs
	tol: 0.0001
	verbose: 0
	warm_start: False


In [60]:
# After using randomsearch --> gridsearch , set the hyparameter c = 0.2 as follow
model_logistic = linear_model.LogisticRegression(penalty='l2', max_iter=200, C=0.2)
model_logistic.fit(xtrain_transform, ytrain)
preds_train = model_logistic.predict(xtrain_transform)
preds_valid = model_logistic.predict(xvalid_transform)
preds_test = model_logistic.predict(xtest_transform)
print('Logistic accuracy train')
print(metrics.accuracy_score(ytrain, preds_train))
print('Logistic accuracy valid')
print(metrics.accuracy_score(yvalid, preds_valid))
print('Logistic accuracy test')
print(metrics.accuracy_score(ytest, preds_test))

Pkl_Filename = "Logistic_cbow.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model_logistic, file)

Logistic accuracy train
0.7891455812247892
Logistic accuracy valid
0.7571707953063885
Logistic accuracy test
0.7542372881355932


### **2.3. Decision Tree BOW** <a class="anchor" id="5"></a>

[Go back to table of contents](#0.1)

In [26]:
model_tree = tree.DecisionTreeClassifier(max_depth=60)
model_tree.fit(xtrain_transform, ytrain)
preds_train = model_tree.predict(xtrain_transform)
preds_valid = model_tree.predict(xvalid_transform)
preds_test = model_tree.predict(xtest_transform)
print('Decision tree accuracy train')
print(metrics.accuracy_score(ytrain, preds_train))
print('Decision tree accuracy valid')
print(metrics.accuracy_score(yvalid, preds_valid))
print('Decision tree accuracy test')
print(metrics.accuracy_score(ytest, preds_test))

Pkl_Filename = "Decision_bow.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model_tree, file)

Decision tree accuracy train
0.8671311575602004
Decision tree accuracy valid
0.7050195567144719
Decision tree accuracy test
0.7131681877444589


### **2.4. KNN bow** <a class="anchor" id="6"></a>

[Go back to table of contents](#0.1)

In [28]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier( n_neighbors = 19,
                              p = 2,
                              weights = 'uniform', n_jobs = -1)

model_knn.fit(xtrain_transform, ytrain)
preds = model_knn.predict(xvalid_transform)
preds_train = model_knn.predict(xtrain_transform)
preds_valid = model_knn.predict(xvalid_transform)
preds_test = model_knn.predict(xtest_transform)
print('KNN accuracy train')
print(metrics.accuracy_score(ytrain, preds_train))
print('KNN accuracy valid')
print(metrics.accuracy_score(yvalid, preds_valid))
print('KNN accuracy test')
print(metrics.accuracy_score(ytest, preds_test))

Pkl_Filename = "KNN_tfidf.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model_knn, file)

KNN accuracy train
0.7250539868801695
KNN accuracy valid
0.7102346805736637
KNN accuracy test
0.7167535853976532


### **2.5. SVM bow** <a class="anchor" id="7"></a>

[Go back to table of contents](#0.1)

In [29]:
from sklearn.svm import SVC
model_svm = SVC(C=2)
model_svm.fit(xtrain_transform, ytrain)
preds_train = model_svm.predict(xtrain_transform)
preds_valid = model_svm.predict(xvalid_transform)
preds_test = model_svm.predict(xtest_transform)
print('SVM accuracy train')
print(metrics.accuracy_score(ytrain, preds_train))
print('SVM accuracy valid')
print(metrics.accuracy_score(yvalid, preds_valid))
print('SVM accuracy test')
print(metrics.accuracy_score(ytest, preds_test))

SVM accuracy train
0.8675793505276453
SVM accuracy valid
0.7679269882659713
SVM accuracy test
0.7636897001303781


# **3. Implement model using TF-IDF** <a class="anchor" id="8"></a>

[Go back to table of contents](#0.1)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
ctv = TfidfVectorizer(  max_features=5000
                      , ngram_range=(1,3)
                      , tokenizer=ViTokenizer.tokenize
                      , stop_words=list(stopword))
                                                            
xtrain, ytrain = convert_df(df_train, dic)
xvalid, yvalid = convert_df(df_valid, dic)
xtest, ytest = convert_df(df_test, dic)

ctv.fit(xtrain)
xtrain_transform, xvalid_transform, xtest_transform = ctv.transform(xtrain), ctv.transform(xvalid), ctv.transform(xtest)

### **3.1 Naive bayes TFIDF** <a class="anchor" id="9"></a>

[Go back to table of contents](#0.1)

In [44]:
model_naivebayes = MultinomialNB()
model_naivebayes.fit(xtrain_transform, ytrain)
preds_train = model_naivebayes.predict(xtrain_transform)
preds_valid = model_naivebayes.predict(xvalid_transform)
preds_test =  model_naivebayes.predict(xtest_transform)
print('Naive BS accuracy train')
print(metrics.accuracy_score(ytrain, preds_train))
print('Naive BS accuracy valid')
print(metrics.accuracy_score(yvalid, preds_valid))
print('Naive BS accuracy test')
print(metrics.accuracy_score(ytest, preds_test))

Naive BS accuracy train
0.7532901438291978
Naive BS accuracy valid
0.7291395045632334
Naive BS accuracy test
0.7438070404172099


### **3.2 Logistic regression TFIDF** <a class="anchor" id="10"></a>

[Go back to table of contents](#0.1)

In [45]:
model_logistic = linear_model.LogisticRegression(penalty='l2', max_iter=200, C=0.6)
model_logistic.fit(xtrain_transform, ytrain)
preds_train = model_logistic.predict(xtrain_transform)
preds_valid = model_logistic.predict(xvalid_transform)
preds_test = model_logistic.predict(xtest_transform)
print('Logistic accuracy train')
print(metrics.accuracy_score(ytrain, preds_train))
print('Logistic accuracy valid')
print(metrics.accuracy_score(yvalid, preds_valid))
print('Logistic accuracy test')
print(metrics.accuracy_score(ytest, preds_test))

Logistic accuracy train
0.809110540683698
Logistic accuracy valid
0.7809647979139505
Logistic accuracy test
0.7897653194263363


### **3.3 Decision Tree TFIDF** <a class="anchor" id="11"></a>

[Go back to table of contents](#0.1)

In [46]:
model_tree = tree.DecisionTreeClassifier(max_depth=60)
model_tree.fit(xtrain_transform, ytrain)
preds_train = model_tree.predict(xtrain_transform)
preds_valid = model_tree.predict(xvalid_transform)
preds_test = model_tree.predict(xtest_transform)
print('Decision tree accuracy train')
print(metrics.accuracy_score(ytrain, preds_train))
print('Decision tree accuracy valid')
print(metrics.accuracy_score(yvalid, preds_valid))
print('Decision tree accuracy test')
print(metrics.accuracy_score(ytest, preds_test))

Decision tree accuracy train
0.976897689768977
Decision tree accuracy valid
0.7050195567144719
Decision tree accuracy test
0.720013037809648


### **3.4 KNN TFIDF** <a class="anchor" id="12"></a>

[Go back to table of contents](#0.1)

In [47]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier( n_neighbors = 19,
                              p = 2,
                              weights = 'uniform', n_jobs = -1)

model_knn.fit(xtrain_transform, ytrain)
preds = model_knn.predict(xvalid_transform)
preds_train = model_knn.predict(xtrain_transform)
preds_valid = model_knn.predict(xvalid_transform)
preds_test = model_knn.predict(xtest_transform)
print('KNN accuracy train')
print(metrics.accuracy_score(ytrain, preds_train))
print('KNN accuracy valid')
print(metrics.accuracy_score(yvalid, preds_valid))
print('KNN accuracy test')
print(metrics.accuracy_score(ytest, preds_test))

KNN accuracy train
0.7839709896915618
KNN accuracy valid
0.7561929595827901
KNN accuracy test
0.7715123859191656


### **3.5 SVM TFIDF** <a class="anchor" id="13"></a>

[Go back to table of contents](#0.1)

In [48]:
model_svm = SVC(C=2)
model_svm.fit(xtrain_transform, ytrain)
preds_train = model_svm.predict(xtrain_transform)
preds_valid = model_svm.predict(xvalid_transform)
preds_test = model_svm.predict(xtest_transform)
print('SVM accuracy train')
print(metrics.accuracy_score(ytrain, preds_train))
print('SVM accuracy valid')
print(metrics.accuracy_score(yvalid, preds_valid))
print('SVM accuracy test')
print(metrics.accuracy_score(ytest, preds_test))

SVM accuracy train
0.9507395183962841
SVM accuracy valid
0.7868318122555411
SVM accuracy test
0.8008474576271186
